In [1]:
#Import necessary documents
import argparse
import http.client
# import httplib2
import os
import random
import time
import datetime
from datetime import datetime, timedelta
from Google import *
import os
from dotenv import load_dotenv
from sqlalchemy import *
import datetime
import re
from StreamMaster import *
from _utils import *
# Connecting to a rasberry pi sql data base, mysql was in this instance

clips_folder = os.environ['CLIPS_FOLDER']
mobile_folder = os.environ['MOBILE_FOLDER']
ml_folder = os.environ['ML_FOLDER']
assets = os.environ['ASSETS']

CLIENT_SECRET_FILE = 'client_secrets.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
# recommend pulling as much access as possible since the Create_Service function will create a pickle token of your authentication allow single authentication for extended period.
SCOPES = ["https://www.googleapis.com/auth/youtube.upload", 'https://www.googleapis.com/auth/youtube']


In [2]:
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

def completeThumbnails(playlist, default_mode = 'default'):
    default = playlist['snippet']['thumbnails'][default_mode]
    checks = ['default', 'medium', 'high', 'standard']
    for chck in checks:
        if not bool(playlist['snippet']['thumbnails'].get(chck)):
            playlist['snippet']['thumbnails'][chck] = default
    return playlist

client_secrets.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube'],)
['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube']
youtube service created successfully


In [2]:
# Key Connection Variables
sql_engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER_NAME']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" ) # Rasberry Pi engine
# lite_engine = create_engine(f"sqlite:///punk-records.db" )

twitch = Twitch(os.environ['TWITCH_APP_ID'], os.environ['TWITCH_APP_SECRET'])

In [13]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)


title_files = []
with Session(sql_engine) as session:
    vid_id, strt, end = session.query(TwitchVideos.id, strtString, endString).where(strtString.ilike('%Brudvaak%')).where(endString.ilike('%Friday Frenzy')).first()
    # session.query(TwitchVideos).where(TwitchVideos.id == vid_id).delete()
    # session.commit()

TypeError: cannot unpack non-iterable NoneType object

In [11]:
vid_id

'1794266158'

In [ ]:
def check_processed_highlights()

In [ ]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)

modes = ['MISSION', 'BOSS', 'FAIL', 'BLOOPER']
# clear_tempfiles()
with Session(sql_engine) as session:
    titles = [ix[0]for ix in session.query(func.distinct(endString)).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).all()]
with Session(sql_engine) as session:
    for title in titles:
        clear_tempfiles()
        title_files = []
        a1 = ffmpeg.input("title_holder.mp4").audio
        ffmpeg.concat(add_transition(create_title(title, 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{title}_ts.mp4", enc_time_base='1/60').run()
        title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{title}_ts.mp4'")
        for mode in modes:
            if bool(session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all()):
                ffmpeg.concat(add_transition(create_title(mode+ " Highlights", 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{mode}_ts.mp4", enc_time_base='1/60').run()
                title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{mode}_ts.mp4'")
                for bossName, fil, end in  session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all():
                    a1 = ffmpeg.input("title_holder.mp4").audio
                    ffmpeg.concat(add_transition(create_title(bossName, 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{bossName}_ts.mp4", enc_time_base='1/60').run()
                    title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{bossName}_ts.mp4'")
                    title_files.append(f"'{os.path.join(input_highlights, fil)}'")
        with open('concat.txt', 'w') as f:
            f.writelines([('file %s\n' % input_path) for input_path in title_files])
        ffmpeg.input('concat.txt', format='concat', safe=0).output(os.path.join(output_highlights, f'{title}.mp4'), c='copy', r=60).run()
